In [ ]:
import json
import requests
import pandas as pd
import folium
from requests.structures import CaseInsensitiveDict
import numpy as np
from datetime import datetime, timedelta
from folium.plugins import MarkerCluster
from scipy.spatial import ConvexHull
from folium.plugins import Draw
from folium import plugins

In [ ]:
url = "https://webapp.tech.musa.co/0.1.1/auth"
payload=json.load(open("login.txt"))
headers = {'content-type': 'application/json'}
r = requests.post(url, data=json.dumps(payload), headers=headers)
token = r.json()

In [ ]:
dias = []
for i in range(8):
    d = (datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d')
    dias.append(d)
dias.sort()

In [ ]:
col = ['id',
     'order',
     'stop_duration',
     'address',
     'coordinates',
     'estimated_time_arrival',
     'estimated_time_departure',
     'collects',
     'destination',
     'storages',
     'stop_point_type',
     'km_distance',
     'check_in_time',
     'check_out_time',
     'about',
     'place_image']


df_gerador = pd.DataFrame()

for l in range(len(dias)):

    FMT = '%H:%M:%S'  
    
    for p in range(1,5):

        url = "https://webapp.tech.musa.co/0.1.1/routes?date=" + dias[l] + "&page=" + str(p) + "&pagination=y&route_type=collect"
        
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        headers["Authorization"] = "Bearer " + token['IdToken']
        
        
        resp = requests.get(url, headers=headers)
        
        if resp.status_code == 200:
            
            rota = resp.json()
            rota = rota['results']

            for i in range(len(rota)):
                
                if rota[i]['stop_points'] != []:
                    df_g = pd.DataFrame(rota[i]['stop_points'])
                    df_g = df_g[df_g['stop_point_type'] == 'generator']
                    lat = []
                    long = []
                    jn = []
                    for _,row in df_g.iterrows():
                        lat.append(float(list(row.coordinates.values())[0]))
                        long.append(list(row.coordinates.values())[1])
                        li = list(row.windows_time.values())
                        li = [x for x in li if x is not None]
                        jn.append(' - '.join(li))
                    df_g['Janelas_de_Coleta'] = jn
                    df_g['latitude'] = lat
                    df_g['longitude'] = long
                    df_g['transportador'] = rota[i]['hauler']['fantasy_name']
                    df_g['roteiro'] = rota[i]['route_name']
                    df_g.drop(columns=col,inplace =True)
                    df_gerador = pd.concat([df_gerador, df_g])
            
df_gerador[['roteiro','data']] = df_gerador['roteiro'].str.split('|', 2, expand=True)
df_gerador.drop(columns=['data','windows_time', 'cnpj'], inplace = True)
df_gerador.drop_duplicates(inplace= True)
df_gerador.reset_index(drop=True, inplace = True)
df_gerador.roteiro = df_gerador.roteiro.str.strip()
df_gerador = df_gerador[(df_gerador['transportador'] != "MUSA Resgate") & (df_gerador['roteiro'] != "Rota MUDA") & (df_gerador['roteiro'] != "Rota Resgate Kata")]
df_gerador.rename(columns = {'name': 'Estabelecimento'}, inplace = True)

In [ ]:
df1 =  pd.read_csv('Color_Roteiro.csv')

In [ ]:
rt = df1['Roteiro'].unique().tolist()
Color = df1['Color'].values.tolist()
H = df1['Horario_roteiro'].values.tolist()

In [ ]:
my_map = folium.Map(location=[-23.5475000, -46.6361100],tiles=None, zoom_start=11, overlay=False)

for i in range(len(rt)):
    globals()['feature_group%s' % i] = folium.FeatureGroup(name= rt[i],overlay=True)
    globals()['feature_group%s' % i].add_to(my_map)

folium.TileLayer(tiles ='Cartodb Positron',overlay=False,name ="Roteiro").add_to(my_map)

folium.LayerControl(autoZIndex=False).add_to(my_map)

In [ ]:
for n in range(len(rt)):
    d = df_gerador[df_gerador['roteiro'] == rt[n]]
    list_of_points = d[['latitude','longitude']].values.tolist()
    form = [list_of_points[i] for i in ConvexHull(list_of_points).vertices]
        
    folium.Polygon(locations=form, 
                    color=Color[n], 
                    fill=True, 
                    fill_color= Color[n], 
                    fill_opacity=0.2,
                   popup=(folium.Popup('Roteiro: ' + rt[n] + '<br> Horário Roteiro: ' + H[n], max_width = 300))
                    ).add_to(globals()['feature_group%s' % n])  

    my_map.add_child(globals()['feature_group%s' % n])

In [ ]:
for n in range(len(rt)):
    for _,row in df_gerador[df_gerador['roteiro'] == rt[n]].iterrows():
        folium.CircleMarker(
            location=[row.latitude,row.longitude],
            radius=5,
            popup =  folium.Popup(row.Estabelecimento + '<br> Janela: ' + 
                                  row.Janelas_de_Coleta + '<br> Roteiro: ' + row.roteiro +'<br> Horário do roteiro: ' + 
                                  H[n], max_width = 300),
            fill=True,
            color=Color[n],
            fill_color=Color[n],
        ).add_to(globals()['feature_group%s' % n])  

    my_map.add_child(globals()['feature_group%s' % n])

In [ ]:
draw = Draw()

draw.add_to(my_map)
plugins.Geocoder(collapsed=True,position = 'topleft' ).add_to(my_map)

In [ ]:
my_map.save('index.html')